# Odkrivanje enačb, 2. del: uporaba predznanja

## 1. Verjetnostne gramatike in knjižnica ProGED
1.1 Za odkrivanje enačb z verjetnostnimi gramatikami bomo uporabili knjižnico ProGED: https://github.com/brencej/ProGED.
Instaliramo jo z ukazom "pip install git+https://github.com/brencej/ProGED". Če se pojavijo težave, ustvari sveže Pythonovo okolje in poskusi instalirati tja.

Poglej si enostaven primer uporabe spodaj. Lahko se poigraš z različnimi nastavitvami in možnostmi. Celoten nabor modelov si lahko ogledaš v "ED.models". 

Tu si lahko ogledas sinakso: https://docs.sympy.org/latest/tutorials/intro-tutorial/gotchas.html#symbols

In [1]:
import numpy as np
import pandas as pd

import ProGED as pg

from vajeED_2_podatki import *

In [2]:
podatki = generiraj_newton(100)

np.random.seed(1)
ED = pg.EqDisco(data=podatki, 
                lhs_vars=["F"],
                rhs_vars=["m", "a"],
                sample_size=10)

ED.generate_models()
ED.fit_models()
ED.get_results()

ModelBox: 1 models
-> [0.999999997446936*a*m], p = 0.00023040000000000002, parse trees = 1, valid = True, error = 9.851219776892273e-10, time = 0.06095409393310547

1.2 Prostor možnih enačb, ki ga želimo preiskovati, definiramo z verjetnostno kontekstno-neodvisno gramatiko. ProGED-u jo podamo v obliki stringa, zapisanega po pravilih knjižnice NLTK:

- produkcijska pravila so oblike "A -> B1 ... Bn [p]", kjer je A neterminal, B1 ... Bn so terminali ali neterminali, p pa je verjetnost pravila
- verjetnosti pravil z istim neterminalom na levi strani se morajo sešteti v 1
- krajši zapis pravil z istim neterminalom na levi strani je "A -> B1 ... Bn [p] | D1 ... Dn [q] | ... " 
- terminalni simboli se označijo z enojnimi narekovaji, npr. "A -> B 'x'"
- v ProGED-u je terminalni simbol "C" poseben - označuje numerične konstante, katerih vrednosti iščemo z optimizacijo

Gramatika za linearne izraze dveh spremenljivk je

E -> E + V | V  
V -> 'x' | 'y'

Uporabi to gramatiko za odkrivanje enačbe $y = x_1 + 3x_2$. Podatke lahko generiraš z "generiraj_linearno". Koraki:
1. zapiši linearno gramatiko v formalizmu NLTK (ne pozabi da so tudi aritmetični operatorji terminalni simboli)
2. naredi ProGED-ov objekt GeneratorGrammar, ki kot argument sprejme string iz točke 1
3. gramatiko iz točke 2 podaj ProGED-ovemu EqDisco kot argument generator
4. generiraj 15 naključnih izrazov z ED.generate_models in si jih oglej
5. kliči še fit\_models in get\_results ter si oglej rezultate

In [26]:
podatki = generiraj_linearno(100)

grammar = "E -> E '+' V [0.6] | V [0.4]\n"
grammar += "V -> 'x1' [0.33] | 'x2' [0.67]"

grammar = pg.GeneratorGrammar(grammar)
ED = pg.EqDisco(data=podatki, 
                lhs_vars=["y"],
                rhs_vars=["x1", "x2"],
                sample_size=15,
                generator=grammar)

ED.generate_models()

ModelBox: 7 models
-> [x1], p = 0.132, parse trees = 30, valid = False
-> [2*x1 + 4*x2], p = 0.0006825639291989762, parse trees = 30, valid = False
-> [x2], p = 0.268, parse trees = 30, valid = False
-> [2*x1], p = 0.026136000000000003, parse trees = 30, valid = False
-> [x1 + 3*x2], p = 0.008575354656000001, parse trees = 30, valid = False
-> [x1 + 2*x2], p = 0.021331727999999998, parse trees = 30, valid = False
-> [x1 + 4*x2], p = 0.0034472925717120005, parse trees = 30, valid = False

Zdaj linearni gramatiki dodaj še numerično konstanto. Kako se spremenijo generirani modeli in rezultati?

In [30]:
podatki = generiraj_linearno(100)

grammar = "E -> E '+' 'C' '*' V [0.6] | 'C' '+' 'C' '*' V [0.4]\n"
grammar += "V -> 'x1' [0.33] | 'x2' [0.67]"

grammar = pg.GeneratorGrammar(grammar)
ED = pg.EqDisco(data=podatki, 
                lhs_vars=["y"],
                rhs_vars=["x1", "x2"],
                sample_size=20,
                generator=grammar)

ED.generate_models()


ModelBox: 3 models
-> [C0*x1 + C1], p = 0.132, parse trees = 33, valid = False
-> [C0*x1 + C1*x2 + C2], p = 0.014730354144000002, parse trees = 33, valid = False
-> [C0*x2 + C1], p = 0.28541056854400004, parse trees = 33, valid = False

1.3. Napiši gramatiko, ki generira polinome, in jo uporabi za odkrivanje energijskega zakona. Verjetno bo potrebno zgenerirati nekaj sto modelov. Če bo imel ProGED težave z generiranjem toliko unikatnih izrazov, mu lahko daš več poskusov za generiranje z nastavitvijo "strategy_settings = {"max_repeat":1000}." 

Na prejšnjih vajah smo videli, da je šum velik trn v peti odkrivanja enačb. Kako vpliva šum na odkrivanje enačb z verjetnostnimi gramatikami?

In [33]:
podatki = generiraj_energijski_zakon(500, sum=0)

grammar = "P -> P '+' M [0.3] | M [0.7]\n"
grammar += "M -> M '*' F [0.5] | 'C' '*' F [0.5]\n"
grammar += "F -> 'm' [0.4] | 'h' [0.3] | 'v' [0.3]"

np.random.seed(1)
grammar = pg.GeneratorGrammar(grammar)
ED = pg.EqDisco(data=podatki, 
                sample_size=500,
                lhs_vars=["E"],
                rhs_vars=["m", "h", "v"],
                strategy_settings = {"max_repeat":1000},
                generator = grammar)
ED.generate_models()
ED.fit_models()
ED.get_results()

Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).


ModelBox: 1 models
-> [9.8099999999971*h*m + 0.499999993727462*m*v**2], p = 2.8349999999999995e-05, parse trees = 434, valid = True, error = 1.5185072515933363e-09, time = 0.33096790313720703

1.4. Loti se še naslednjih enačb:
1. projekcija na y os pri kroženju $y = r\sin(\omega t)$, kjer je konstanta (*generiraj_krozenje*).  
2. Lorenzov popravek $\gamma = \sqrt{1 - v^2/c^2}$, kjer je c hitrost svetlobe, približno $3\cdot10^8 m/s$ (*generiraj_lorenz*)

Za vsako od enačb napiši gramatiko, ki jo lahko odkrije. Premisli tudi, kakšne verjetnosti produkcij bi bile smiselne. Poskusi čim bolj omejiti prostor enačb, pri čemer naj ima gramatika še vedno vsaj eno rekurzivno pravilo. Gramatike lahko preizkusiš tudi na podatkih.


In [39]:
gramatika1 = "E -> E '*' F [0.4] | F [0.6] \n"
gramatika1 += "F -> V [0.6] | T '(' M ')' [0.4] \n"
gramatika1 += "T -> 'sin' [0.4] | 'cos' [0.4] | 'tan' [0.2] \n"
gramatika1 += "M -> M '*' V [0.4] | V [0.6] \n"
gramatika1 += "V -> 'r' [0.4] | 't' [0.4] | 'C' [0.2]"

gramatika1 = pg.GeneratorGrammar(gramatika1)
podatki = generiraj_krozenje(100)
ED = pg.EqDisco(data=podatki, 
                sample_size=50,
                lhs_vars=["y"],
                rhs_vars=["r", "t"],
                generator = gramatika1)
ED.generate_models()
ED.fit_models()
ED.get_results(5)

ModelBox: 5 models
-> [1.05126239695454*r**2*cos(3.24191883284816*r*t**2)], p = 2.0873541058560008e-08, parse trees = 439, valid = True, error = 0.16078119825087922, time = 0.08463096618652344
-> [cos(2.49392835792852*r*t)*tan(r)], p = 2.264924160000001e-06, parse trees = 439, valid = True, error = 0.25474201880142033, time = 0.008666038513183594
-> [-0.328774073439291*cos(6.55645272411799*t)], p = 1.4155776000000007e-05, parse trees = 439, valid = True, error = 0.2885593367369822, time = 0.05254197120666504
-> [0.392942888046269*r*cos(t)], p = 0.00010616832000000001, parse trees = 439, valid = True, error = 0.3204031830686981, time = 0.012373208999633789
-> [-0.4279740520664*r*cos(7.85078232102086*r*t)], p = 1.0871635968000005e-07, parse trees = 439, valid = True, error = 0.32779614935450196, time = 0.05377078056335449

In [53]:
gramatika2 = "E -> E '+' F [0.2] | E '-' F [0.2] | F [0.6] \n"
gramatika2 += "F -> F '*' T [0.2] | F '/' T [0.2] | T [0.6] \n"
gramatika2 += "T -> R [0.3] | V [0.4] | 'C' [0.3] \n"
gramatika2 += "R -> '(' E ')' [0.3] | 'sqrt' '(' E ')' [0.3] | 'exp' '(' E ')' [0.2] | 'log' '(' E ')' [0.2] \n"
gramatika2 += "V -> 'v' [1.0]"
gramatika2 = pg.GeneratorGrammar(gramatika2)

podatki = generiraj_lorenz(100)
ED = pg.EqDisco(data=podatki, 
                sample_size=50,
                lhs_vars=["gama"],
                rhs_vars=["v"],
                generator = gramatika2)
ED.generate_models()
ED.fit_models()
ED.get_results()

<lambdifygenerated-36717>:2: RuntimeWarning: invalid value encountered in sqrt
  return -3.64602513252549*sqrt(0.0710681061287593 - v) + 2.62954062636809 + 9.39237565262157/(v**4*log(sqrt(v**2) - 4.0106708960691))
<lambdifygenerated-36718>:2: RuntimeWarning: invalid value encountered in sqrt
  return -3.64602513252549*sqrt(0.0710681061287593 - v) + 2.62954064126926 + 9.39237565262157/(v**4*log(sqrt(v**2) - 4.0106708960691))
<lambdifygenerated-36719>:2: RuntimeWarning: invalid value encountered in sqrt
  return -3.64602513252549*sqrt(0.0710681061287593 - v) + 2.62954062636809 + 9.39237566752273/(v**4*log(sqrt(v**2) - 4.0106708960691))
<lambdifygenerated-36720>:2: RuntimeWarning: invalid value encountered in sqrt
  return -3.64602513252549*sqrt(0.0710681061287593 - v) + 2.62954062636809 + 9.39237565262157/(v**4*log(sqrt(v**2) - 4.01067088116794))
<lambdifygenerated-36721>:2: RuntimeWarning: invalid value encountered in sqrt
  return -3.64602513048201*sqrt(0.0710681062084221 - v) + 2.6295

Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).


<lambdifygenerated-37778>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.0289548029149311*v**2))
<lambdifygenerated-37779>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.0289547880137699*v**2))
<lambdifygenerated-37790>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.0143627430149973*v**2))
<lambdifygenerated-37791>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.0143627281138361*v**2))
<lambdifygenerated-37796>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.00706671307517759*v**2))
<lambdifygenerated-37797>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.00706669817401639*v**2))
<lambdifygenerated-37802>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.00341869810780482*v**2))
<lambdifygenerated-37803>:2: RuntimeWarning: invalid value encountered in log
  return log(log(-0.00341868320664362*v**2))
<lambdifygenerated-3

Model skipped. More model parameters than allowed (check max constant).


<lambdifygenerated-38415>:2: RuntimeWarning: overflow encountered in exp
  return -4.24674749662001*v - exp(v) - 6.73549479773615
<lambdifygenerated-38416>:2: RuntimeWarning: overflow encountered in exp
  return -4.24674748171884*v - exp(v) - 6.73549479773615
<lambdifygenerated-38417>:2: RuntimeWarning: overflow encountered in exp
  return -4.24674749662001*v - exp(v) - 6.73549478283499


ModelBox: 1 models
-> [1.14974026502787 - 2.37135586589332e-6*v], p = 1.4929919999999999e-05, parse trees = 442, valid = True, error = 0.08338169511209467, time = 0.18238091468811035

## 2. Preštevanje dreves in verjetnosti

Za linearno gramatiko iz 1.2 ter vsako od gramatik iz 1.4. nariši:
1. graf števila dreves z dano višino
2. graf verjetnosti vseh dreves do vključno dane višine

Če želiš, lahko sam implementiraš rekurzivne enačbe s predavanj. Alternativno lahko uporabiš ProGED-ovi metodi "grammar.count_trees" in "grammar.count_coverage". Obema je treba podati začetni neterminalni simbol, ki mora biti objekt nltk.Nonterminal.



In [ ]:
from nltk import Nonterminal
import matplotlib.pyplot as plt



## 3. Merske enote

Upoštevanje predznanja o merskih enotah spremenljivk nam pogosto lahko močno zmanjša prostor enačb. 
ProGED nam to omogoča preko formalizma atributnih gramatik, ki jih pretvorimo v navadne konteksno-neodvisne gramatike. Za pogoste tipe gramatik ProGED že implementira njihove dimenzijsko-pravilne različice.
Enote predstavimo z vektorji v izbrani bazi osnovnih enot (recimo SI). 

3.1 Poglej si spodnji primer za Newtonov zakon ter dopolni manjkajoče!

Plonkec iz osnovne šole:
- Newtonov zakon $ F=m a$
- F je sila, z enoto kg*m/s^2
- m je masa, z enoto kg
- a je pospešek, z enoto m/s^2

In [ ]:
from ProGED.generators.dimensions import construct_dimensional_polynomial_grammar

podatki = generiraj_newton(100)

spremenljivke = ["m", "a"]
simboli_enot = ["m", "s", "kg"]
enote_na_desni = # DOPOLNI
enota_na_levi = # DOPOLNI

grammar = construct_dimensional_polynomial_grammar(variables = spremenljivke,
                                        units = enote_na_desni,
                                        target_variable_unit = enota_na_levi,
                                        unit_symbols = simboli_enot)

grammar

Vidimo, da ima gramatika veliko več simbolov in produkcij, kot tiste, ki smo jih pisali na roko. Zdaj uporabimo EqDisco in zgenerirajmo množico enačb.

In [ ]:
ED = pg.EqDisco(data=podatki, 
                lhs_vars=["F"],
                rhs_vars=["m", "a"],
                sample_size=10,
                generator = grammar)

ED.generate_models()


3.2. Poglejmo še pod krov, torej atributno gramatiko, ki izraža pravila za računanje z merskimi enotami. Oglej si spodnjo atributno gramatiko za dimenzijsko-pravilne polinome ter premisli, kako bi napisal dimenzijsko-pravilno atributno verzijo gramatik, ki si jih sestavil v nalogi 1.4. 

In [ ]:
from ProGED.generators.dimensions import dimensional_attribute_grammar_to_pcfg
from nltk import PCFG

prods = []
prods += [["P -> P '+' M", 0.4, ["uP1 - uP2", "uP1 - uM1"]]]
prods += [["P -> M", 0.6, ["uP1 - uM1"]]]
prods += [["M -> M '*' V", 0.4, ["-uM1 + uM2 + uV1"]]]
prods += [["M -> V", 0.4, ["uM1 - uV1"]]]
prods += [["M -> 'C'", 0.2, ["uM1"]]]

pcfg_start, pcfg_prods = dimensional_attribute_grammar_to_pcfg(prods, 
                                                               spremenljivke, 
                                                               enote_na_desni, 
                                                               enota_na_levi, 
                                                               append_vars = True, 
                                                               unit_symbols = simboli_enot)
grammar = pg.GeneratorGrammar(PCFG(pcfg_start, pcfg_prods))
grammar